<a href="https://colab.research.google.com/github/smjenski-izvjestaji/izvjestaji/blob/main/source_kod.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Step 1: Install required packages
!pip install pandas openpyxl requests

# Step 2: Upload the Excel template
from google.colab import files
uploaded = files.upload()

import pandas as pd
import requests
import base64
import time as systime
import ast
from openpyxl import load_workbook
from openpyxl import Workbook
from openpyxl.cell.cell import MergedCell
from openpyxl.utils import get_column_letter
from datetime import datetime, time
from openpyxl.styles import Alignment, Font, Border  # remove when going back to .py version
from copy import copy
import requests

from datetime import datetime, time, timedelta
from zoneinfo import ZoneInfo
import textwrap
import os
import smtplib
from email.message import EmailMessage


import openpyxl
from datetime import datetime
from pathlib import Path
import subprocess


# Replace with your actual timezone (e.g., 'Europe/Belgrade', 'Europe/Zagreb')
local_timezone = ZoneInfo('Europe/Zagreb')

# Get local time
now = datetime.utcnow()
now_local = now.replace(tzinfo=ZoneInfo('UTC')).astimezone(local_timezone)

current_time = now_local.time()

# Determine shift and date based on local time
# vrijeme je u UTC+0 by default a ne CEST (UTC+2) PAZI!!!
# zato smo uveli now_local
if time(5, 30) <= current_time < time(13, 30):
    shift = 1
    selected_date = now_local.date()
elif time(13, 30) <= current_time < time(21, 30):
    shift = 2
    selected_date = now_local.date()
else:
    shift = 3
    selected_date = (now_local - timedelta(days=1)).date()  # Use previous day

# Station mapping
station_map = {"RGB": 1, "PET1": 2, "PET2": 3}

# Use today's date and current time in UTC
#now = datetime.now()
#selected_date = now.date()
#current_time = now.time()

# Format date for filename
date_str = selected_date.strftime('%d%m%Y')
created_files=[]
# Iterate over each production line
for line_input, station_id in station_map.items():
    print(f"Processing line: {line_input} (Station ID: {station_id})")

    # Replace with your actual file URL
    github_url = "https://raw.githubusercontent.com/<smjenski-izvjestaji>/<izvjestaji>/<main>/path/to/Shift_report_template.xlsx"
    #promijeni username, ime repozitorija, path, ime xlsx filea
    local_filename = "Shift_report_template.xlsx"

    # Download and save the file
    with open(local_filename, "wb") as f:
      response = requests.get(github_url)
      if response.status_code == 200:
          f.write(response.content)
          print(f"✅ Downloaded: {local_filename}")
      else:
          raise Exception(f"❌ Failed to download file: {response.status_code}")
    filename=local_filename
    # 🧠 Step 3: Load workbook and sheet
    #filename = next(iter(uploaded))
    wb = load_workbook(filename)
    #systime.sleep(10)  # Optional delay
    sheet_814 = wb['Sheet1']

    # 🔐 Step 4: Prepare API credentials
    api_key = os.getenv("API_KEY")         # 🔁 Replace with your API key
    secret_key = os.getenv("SECRET_KEY")   # 🔁 Replace with your secret key

    credentials = f"{api_key}:{secret_key}"
    encoded_credentials = base64.b64encode(credentials.encode()).decode()

    headers = {
    "Authorization": f"Basic {encoded_credentials}"
    }

    """
    # 🏭 Ask user for production line
    line_input = input("Enter production line (PET1, PET2, or RGB): ").strip().upper().replace(" ", "")
    station_map = {"RGB": 1, "PET1": 2, "PET2": 3}

    if line_input not in station_map:
    raise ValueError("Invalid line. Use RGB, PET1, or PET2.")

    station_id = station_map[line_input]

    # 📅 Ask user for date
    selected_date_input = input("Enter the date to retrieve checklists and stops (DD.MM.YYYY): ").strip()
    try:
        selected_date = datetime.strptime(selected_date_input, "%d.%m.%Y").date()
    except ValueError:
        raise ValueError("Invalid date format. Please use DD.MM.YYYY.")


    # 👥 Ask user for shift number
    shift_input = input("Enter shift number (1, 2, or 3): ").strip()
    if shift_input not in ['1', '2', '3']:
        raise ValueError("Invalid shift. Please enter 1, 2, or 3.")
    shift = int(shift_input)
    shift_label = {1: "1st", 2: "2nd", 3: "3rd"}[shift]
    """
    """
    # Station map
    station_map = {"RGB": 1, "PET1": 2, "PET2": 3}

    # ⏰ Automatically determine shift based on current time
    # 📅 Automatically use today or yesterday's date based on shift
    current_time = datetime.now().time()

    if time(3, 30) <= current_time < time(11, 30):
        shift = 1
        selected_date = datetime.today().date
    elif time(11, 30) <= current_time < time(19, 30):
        shift = 2
        selected_date = datetime.today().date
    else:
        shift = 3
        selected_date = (datetime.today() - timedelta(days=1)).date()

    shift_label = {1: "1st", 2: "2nd", 3: "3rd"}[shift]

    # delay se moze dodati samo time da odgodimo vrijeme za npr pola sata ovdje
    # ali odma u slijedecoj liniji ostavimo isto zbog unosa u tablicu
    """
    shift_label = {1: "1st", 2: "2nd", 3: "3rd"}[shift]

    #  Define shift time windows for start and end
    def get_shift_bounds(shift, date):
        if shift == 1:
            return time(5, 30), time(13, 30)
        elif shift == 2:
            return time(13, 30), time(21, 30)
        elif shift == 3:
            # Third shift spans 2 dates
            return time(21, 30), time(5, 30)
        else:
            raise ValueError("Invalid shift number")

#
    # 🧾 Write date and shift to cells D9 and F9
    def set_left(cell, value):
        cell.value = value
        cell.alignment = Alignment(horizontal='left')

    def set_left_font48(cell, value):
        cell.value = value
        cell.alignment = Alignment(horizontal='left')
        cell.font = Font(name="Arial", size=48)

    def set_center(cell, value):
        cell.value = value
        cell.alignment = Alignment(horizontal='center')

    def set_right(cell, value):
        cell.value = value
        cell.alignment = Alignment(horizontal='right')

    set_left(sheet_814["D9"], selected_date.strftime("%d.%m.%Y"))
    set_left(sheet_814["F9"], str(shift))

    # write other info in header (punjac, linija, ...)

    if line_input == "PET1":
        set_center(sheet_814["C6"], "Punjač O+H")
        set_center(sheet_814["E6"], "PUNJENJE PET BOCA")
        set_center(sheet_814["Q6"], "LINIJA 866 (PET1)")

    elif line_input == "PET2":
        set_center(sheet_814["C6"], "Punjač Krones")
        set_center(sheet_814["E6"], "PUNJENJE PET BOCA")
        set_center(sheet_814["Q6"], "LINIJA 814 (PET2)")

    elif line_input == "RGB":
        set_center(sheet_814["C6"], "Punjač Krones")
        set_center(sheet_814["E6"], "PUNJENJE RGB BOCA")
        set_center(sheet_814["Q6"], "LINIJA 865 (RGB)")



    # 📡 Step 5: Fetch checklist data from API
    checklist_url = f"https://api.evocon.com/api/reports/checklists_json?startTime=2025-03-01&endTime=2050-01-01&stationId={station_id}"
    response = requests.get(checklist_url, headers=headers)
    if response.status_code != 200:
        raise Exception(f"Checklist API error: {response.status_code}")

    checklist_df = pd.DataFrame(response.json())

    # 🧾 Step 6: Normalize checklist data
    def clean_time(t):
        if pd.isna(t): return None
        t = str(t).replace(":", "").strip()
        return t.zfill(4)[-4:]

    # Handle stringified lists in 'itemresult'
    def extract_single_value(x):
        if isinstance(x, str) and x.startswith('[') and x.endswith(']'):
            try:
                parsed = ast.literal_eval(x)
                if isinstance(parsed, list) and len(parsed) == 1:
                    return str(parsed[0])
            except:
                return x
        return str(x).strip()

    checklist_df['clean_duetime'] = checklist_df['duetime'].apply(clean_time)
    checklist_df['duetime_dt'] = pd.to_datetime(checklist_df['duetime'], format='%H:%M', errors='coerce').dt.time
    checklist_df['date'] = pd.to_datetime(checklist_df['date'], errors='coerce').dt.date
    checklist_df['itemname'] = checklist_df['itemname'].astype(str).str.strip().str.lower()
    checklist_df['result'] = checklist_df['result'].astype(str).str.strip().str.lower()
    checklist_df['itemresult'] = checklist_df['itemresult'].apply(extract_single_value)

    # Apply shift filter
    if shift == 1:
        time_filter = (checklist_df['date'] == selected_date) & \
                      (checklist_df['duetime_dt'] >= time(6, 0)) & \
                      (checklist_df['duetime_dt'] <= time(14, 0))
    elif shift == 2:
        time_filter = (checklist_df['date'] == selected_date) & \
                      (checklist_df['duetime_dt'] > time(14, 0)) & \
                      (checklist_df['duetime_dt'] <= time(22, 0))
    elif shift == 3:
        next_day = selected_date + pd.Timedelta(days=1)
        time_filter = (
            ((checklist_df['date'] == selected_date) & (checklist_df['duetime_dt'] > time(22, 0))) |
            ((checklist_df['date'] == next_day) & (checklist_df['duetime_dt'] <= time(5, 46)))
        )

    checklist_df = checklist_df[time_filter]

    # Map parameter names from D17–D27
    param_row_map = {}
    for row in range(17, 28):
        val = sheet_814[f"D{row}"].value
        if val:
            param_row_map[val.strip().lower()] = row

    # Match headers in row 16
    header_map = {}
    for col in sheet_814.iter_cols(min_row=16, max_row=16):
        cell = col[0]
        if cell.value:
            values = [p.strip().replace(":", "") for p in str(cell.value).split('/')]
            if len(values) == 3:
                header_map[col[0].column_letter] = {
                    'morning': values[0].zfill(4),
                    'afternoon': values[1].zfill(4),
                    'night': values[2].zfill(4)
                }

    # Write checklist values
    for _, row in checklist_df.iterrows():
        if row['result'] != 'successful':
            continue

        param_name = row['itemname']
        result = row['itemresult']
        duetime = row['duetime_dt']
        duetime_str = row['clean_duetime']

        if pd.isna(duetime) or param_name not in param_row_map:
            continue

        shift_key = None
        if time(6, 15) <= duetime <= time(13, 45):
            shift_key = 'morning'
        elif time(14, 15) <= duetime <= time(21, 45):
            shift_key = 'afternoon'
        elif duetime >= time(22, 15) or duetime <= time(5, 45):
            shift_key = 'night'

        target_column = None
        for col_letter, shift_times in header_map.items():
            if shift_times.get(shift_key) == duetime_str:
                target_column = col_letter
                break

        if not target_column:
            continue

        row_number = param_row_map[param_name]
        cell = sheet_814[f"{target_column}{row_number}"]
        if not isinstance(cell, MergedCell):
            set_left(cell, result)

    # 🌐 API 2: Stop reasons
    losses_url = f"https://api.evocon.com/api/reports/losses_json?stationId={station_id}&startTime=2025-03-01&endTime=2050-01-01"
    response = requests.get(losses_url, headers=headers)
    if response.status_code != 200:
        raise Exception(f"Losses API error: {response.status_code}")

    losses_df = pd.DataFrame(response.json())

    losses_df['shiftStart'] = pd.to_datetime(losses_df['shiftStart'], errors='coerce')
    losses_df['start'] = pd.to_datetime(losses_df['start'], errors='coerce')
    losses_df['end'] = pd.to_datetime(losses_df['end'], errors='coerce')

    filtered_losses = losses_df[
        (losses_df['shiftStart'].dt.date == selected_date) &
        (losses_df['shiftName'] == shift_label)
    ]

# Write stops to 814 starting from row 32
# New scrit with joined stops

    start_row = 32
    processed_join_ids = set()

    for idx, row in filtered_losses.iterrows():
        join_id = row.get('stopJoinId')

        if join_id and str(join_id).strip() != "":
            if join_id in processed_join_ids:
                continue
            processed_join_ids.add(join_id)

            # ✅ get full join group from losses_df (not just filtered)
            group = losses_df[losses_df['stopJoinId'] == join_id].copy()
            group = group.sort_values(by='start')

            total_duration = 0
            for _, sub in group.iterrows():
                if pd.notna(sub['start']) and pd.notna(sub['end']):
                    total_duration += (sub['end'] - sub['start']).total_seconds() / 60
            total_duration = round(total_duration, 0)

            location = group['location'].dropna().astype(str).str.strip().iloc[0] if not group['location'].dropna().empty else ''
            stop_group = group['stopGroup'].dropna().astype(str).str.strip().iloc[0] if not group['stopGroup'].dropna().empty else ''
            stop = str(group['stop'].iloc[0]).strip()
            comment = str(group['comment'].iloc[0]).strip()
            combined = f"{stop} ({comment})".strip()

            if start_row >= 49:
                sheet_814.insert_rows(start_row)

                # 🧽 Unmerge any cells that were auto-merged in the inserted row
                for merged_range in list(sheet_814.merged_cells.ranges):
                    if merged_range.min_row == start_row:
                        try:
                            sheet_814.unmerge_cells(str(merged_range))
                        except KeyError:
                            pass # merged cell may already have been cleared

                # 🎨 Copy style from row 49 to the new row

                for col in range(1, sheet_814.max_column + 1):
                    source_cell = sheet_814.cell(row=49, column=col)
                    target_cell = sheet_814.cell(row=start_row, column=col)

                    # Copy formatting if any
                    if source_cell.has_style:
                        target_cell.font = copy(source_cell.font)
                        target_cell.border = copy(source_cell.border)
                        target_cell.fill = source_cell.fill
                        target_cell.number_format = source_cell.number_format
                        target_cell.protection = source_cell.protection
                        target_cell.alignment = source_cell.alignment

                # Set row height for larger font
                sheet_814.row_dimensions[start_row].height = 90


            set_left_font48(sheet_814[f"E{start_row}"], "joined stops")
            set_left_font48(sheet_814[f"F{start_row}"], "joined stops")
            set_left_font48(sheet_814[f"G{start_row}"], total_duration)
            set_left_font48(sheet_814[f"H{start_row}"], location)
            set_left_font48(sheet_814[f"K{start_row}"], stop_group)
            set_left_font48(sheet_814[f"N{start_row}"], combined)

            start_row += 1

        else:
            # ✅ true individual stop (no join ID or empty)
            if start_row >= 49:
                sheet_814.insert_rows(start_row)

                # 🧽 Unmerge any cells that were auto-merged in the inserted row
                for merged_range in list(sheet_814.merged_cells.ranges):
                    if merged_range.min_row == start_row:
                        try:
                            sheet_814.unmerge_cells(str(merged_range))
                        except KeyError:
                            pass # merged cell may already have been cleared

            # 🎨 Copy style from row 49 to the new row
                for col in range(1, sheet_814.max_column + 1):
                    source_cell = sheet_814.cell(row=49, column=col)
                    target_cell = sheet_814.cell(row=start_row, column=col)

                    # Copy formatting if any
                    if source_cell.has_style:
                        target_cell.font = copy(source_cell.font)
                        target_cell.border = copy(source_cell.border)
                        target_cell.fill = copy(source_cell.fill)
                        target_cell.number_format = copy(source_cell.number_format)
                        target_cell.protection = copy(source_cell.protection)
                        target_cell.alignment = copy(source_cell.alignment)

                # Set row height for larger font
                sheet_814.row_dimensions[start_row].height = 90

            start_time = row['start'].strftime('%H:%M') if pd.notna(row['start']) else ''
            end_time = row['end'].strftime('%H:%M') if pd.notna(row['end']) else ''
            duration = ''
            if pd.notna(row['start']) and pd.notna(row['end']):
                duration = (row['end'] - row['start']).total_seconds() / 60
                duration = round(duration, 0)

            location = str(row.get('location', '')).strip()
            stop_group = str(row.get('stopGroup', '')).strip()
            stop = str(row.get('stop', '')).strip()
            comment = str(row.get('comment', '')).strip()
            combined = f"{stop} ({comment})".strip()

            set_left_font48(sheet_814[f"E{start_row}"], start_time)
            set_left_font48(sheet_814[f"F{start_row}"], end_time)
            set_left_font48(sheet_814[f"G{start_row}"], duration)
            set_left_font48(sheet_814[f"H{start_row}"], location)
            set_left_font48(sheet_814[f"K{start_row}"], stop_group)
            set_left_font48(sheet_814[f"N{start_row}"], combined)

            start_row += 1

    # 🔁 Speed losses API
    speed_url = f"https://api.evocon.com/api/reports/speedlosses_json?stationId={station_id}&startTime=2025-03-01&endTime=2050-01-01"
    response = requests.get(speed_url, headers=headers)
    if response.status_code != 200:
        raise Exception(f"Speed losses API error: {response.status_code}")

    speed_df = pd.DataFrame(response.json())
    speed_df['start'] = pd.to_datetime(speed_df['start'], errors='coerce')
    speed_df['shiftName'] = speed_df['shiftName'].astype(str).str.strip()
    speed_df['performanceLossNotes'] = speed_df['performanceLossNotes'].astype(str).fillna('').str.strip()
    speed_df['stopMinutes'] = pd.to_numeric(speed_df['stopMinutes'], errors='coerce').fillna(0)

    # Filter by selected date and shift
    if shift == 3:
        next_day = selected_date + pd.Timedelta(days=1)
        speed_filtered = speed_df[
            ((speed_df['start'].dt.date == selected_date) & (speed_df['start'].dt.time > time(21, 30))) |
            ((speed_df['start'].dt.date == next_day) & (speed_df['start'].dt.time <= time(5, 30)))
        ]
    else:
        speed_filtered = speed_df[
            (speed_df['start'].dt.date == selected_date) &
            (speed_df['shiftName'] == shift_label)
        ]

    # Group by comment (performanceLossNotes)
    grouped_speed = speed_filtered.groupby('performanceLossNotes')

    for comment, group in grouped_speed:
        total_duration = round(group['stopMinutes'].sum(), 0)

        if start_row >= 49:
            sheet_814.insert_rows(start_row)
            for merged_range in list(sheet_814.merged_cells.ranges):
                if merged_range.min_row == start_row:
                    try:
                        sheet_814.unmerge_cells(str(merged_range))
                    except KeyError:
                        pass
            for col in range(1, sheet_814.max_column + 1):
                source_cell = sheet_814.cell(row=49, column=col)
                target_cell = sheet_814.cell(row=start_row, column=col)
                if source_cell.has_style:
                    target_cell.font = copy(source_cell.font)
                    target_cell.border = copy(source_cell.border)
                    target_cell.fill = copy(source_cell.fill)
                    target_cell.number_format = copy(source_cell.number_format)
                    target_cell.protection = copy(source_cell.protection)
                    target_cell.alignment = copy(source_cell.alignment)

        # Write values to sheet
        set_left(sheet_814[f"E{start_row}"], "gubitak brzine")
        set_left(sheet_814[f"F{start_row}"], "gubitak brzine")
        set_left(sheet_814[f"G{start_row}"], total_duration)
        set_left(sheet_814[f"K{start_row}"], "gubitak brzine")
        set_left(sheet_814[f"N{start_row}"], comment)

        start_row += 1



    # 🚀 STEP: Fill product info (product name, SKU, start/end time of runs) into Sheet1


    # 🧩 1. Get shift time boundaries
    shift_start_time, shift_end_time = get_shift_bounds(shift, selected_date)

    # 👀 2. Get first product info from the first stop
    first_stop_row = filtered_losses.iloc[0]
    first_product_name = str(first_stop_row.get("productName", "")).strip()
    first_product_sku = str(first_stop_row.get("productSku", "")).strip()

    # 🖋️ Fill F10 (SKU), F11 (product), and J10 (shift start time) initially
    sheet_814["F10"].value = first_product_sku
    sheet_814["F11"].value = first_product_name
    sheet_814["J10"].value = shift_start_time.strftime("%H:%M")

    # ⏳ 3. Fetch production runs and filter those within this shift
    prod_url = f"https://api.evocon.com/api/reports/production_runs_json?startTime=2025-03-01&endTime=2050-01-01&stationId={station_id}"
    prod_df = pd.DataFrame(requests.get(prod_url, headers=headers).json())

    # Normalize times and dates
    prod_df['startDate'] = pd.to_datetime(prod_df['startDate'], errors='coerce').dt.date
    prod_df['startTime'] = pd.to_datetime(prod_df['startTime'], errors='coerce')
    prod_df['endTime'] = pd.to_datetime(prod_df['endTime'], errors='coerce')

    # For 3rd shift, we must span dates
    if shift == 3:
        next_day = selected_date + pd.Timedelta(days=1)
        shift_filter = (
            ((prod_df['startDate'] == selected_date) & (prod_df['startTime'].dt.time >= shift_start_time)) |
            ((prod_df['startDate'] == next_day) & (prod_df['startTime'].dt.time <= shift_end_time))
        )
    else:
        shift_filter = (
            (prod_df['startDate'] == selected_date) &
            (prod_df['startTime'].dt.time >= shift_start_time) &
            (prod_df['startTime'].dt.time <= shift_end_time)
        )

    prod_filtered = prod_df[shift_filter].sort_values(by="startTime").reset_index(drop=True)

    # 🧠 4. Build initial production run list from stop data
    production_runs = [{
        "sku": first_product_sku,
        "product": first_product_name,
        "start_time": shift_start_time.strftime("%H:%M"),
        "end_time": None  # to be filled later
    }]

    # 🔍 5. Detect "Changeover and CIP" stops to determine end/start transitions
    changeover_rows = filtered_losses[filtered_losses["stopGroup"] == "Changeover and CIP"].copy()
    changeover_rows = changeover_rows.sort_values(by="start").reset_index(drop=True)

    # Assign end time of first run
    if not changeover_rows.empty and pd.notna(changeover_rows.iloc[0]["start"]):
        production_runs[0]["end_time"] = changeover_rows.iloc[0]["start"].strftime("%H:%M")
    else:
        production_runs[0]["end_time"] = shift_end_time.strftime("%H:%M")


    # 🔁 6. Append production runs from prod_filtered (with proper changeover sequencing)
    changeover_rows = changeover_rows.reset_index(drop=True)  # Safe for iloc
    all_stops = filtered_losses.reset_index(drop=True)        # Ensure same for full stops list

    changeover_idx = 0
    last_valid_end = None

    for i, row in prod_filtered.iterrows():
        sku = str(row.get("sku", "")).strip()
        product = str(row.get("product", "")).strip()

        # 🧩 Find the last "Changeover and CIP" in a consecutive sequence
        cip_end_time = None
        while changeover_idx < len(changeover_rows):
            current_cip = changeover_rows.iloc[changeover_idx]
            changeover_idx += 1

            # If last or next stop is not "Changeover and CIP", this is the last in sequence
            if changeover_idx == len(changeover_rows):
                cip_end_time = current_cip.get("end")
                break

            next_cip = changeover_rows.iloc[changeover_idx]
            curr_idx_in_all = all_stops[all_stops['start'] == current_cip['start']].index[0]

            if curr_idx_in_all + 1 < len(all_stops):
                next_stop_group = all_stops.iloc[curr_idx_in_all + 1].get("stopGroup", "")
                if next_stop_group != "Changeover and CIP":
                    cip_end_time = current_cip.get("end")
                    break

        # 🕒 Start time = end of last CIP, or start of shift
        if pd.notna(cip_end_time):
            start_time = cip_end_time
        else:
            start_time = row.get("startTime") or shift_start_time

        start_str = start_time.strftime("%H:%M") if pd.notna(start_time) else ""

        # 🕛 End time = next CIP start, or shift end
        if changeover_idx < len(changeover_rows):
            next_cip_start = changeover_rows.iloc[changeover_idx]["start"]
            end_str = next_cip_start.strftime("%H:%M") if pd.notna(next_cip_start) else shift_end_time.strftime("%H:%M")
        else:
            end_str = shift_end_time.strftime("%H:%M")

        production_runs.append({
            "sku": sku,
            "product": product,
            "start_time": start_str,
            "end_time": end_str
        })


    #version of insert rows and writing with font and size formatting and with row height adjustment


    # 🔠 Font config
    prod_font = Font(name="Arial", size=48)
    bold_font = Font(bold=True)

    # 🧱 Insert new rows (if more than one product)
    insert_point = 10
    extra_runs = len(production_runs) - 1

    if extra_runs > 0:
        for _ in range(extra_runs * 2):
            sheet_814.insert_rows(insert_point)

    # 🧾 content to be written in new cells inserted
    title_map = {
        "E": "Šifra:",
        "E_lower": "Naziv proizvoda:",
        "H": "Proizvodnja od (prva boca kontinuirano na punjaču):",
        "H_lower": "Proizvodnja do (zadnja boca na punjaču):",
    }

    # 🧬 Apply values and formatting row by row
    for i, run in enumerate(production_runs):
        row_top = 10 + (i * 2)
        row_bottom = row_top + 1

        # Copy titles to column E and H
        sheet_814[f"E{row_top}"].value = title_map["E"]
        sheet_814[f"E{row_bottom}"].value = title_map["E_lower"]
        sheet_814[f"H{row_top}"].value = title_map["H"]
        sheet_814[f"H{row_bottom}"].value = title_map["H_lower"]

        for col in ['E', 'F', 'H', 'J']:
            for r in [row_top, row_bottom]:
                cell = sheet_814[f"{col}{r}"]
                cell.font = prod_font
                cell.alignment = Alignment(horizontal='left', wrap_text=True)

        # Fill values for product info
        set_left(sheet_814[f"F{row_top}"], run["sku"])
        set_left(sheet_814[f"F{row_bottom}"], run["product"])
        set_left(sheet_814[f"J{row_top}"], run["start_time"])
        set_left(sheet_814[f"J{row_bottom}"], run["end_time"])
        # ovdje je u og kodu bilo height = 159
        # Dynamically adjust row height based on the maximum number of lines in the cells
        def get_max_lines(row_num, columns):
            max_lines = 1
            for col in columns:
                cell_value = str(sheet_814[f"{col}{row_num}"].value or "")
                lines = cell_value.count('\n') + 1
                max_lines = max(max_lines, lines)
                        # Ensure wrapText is enabled
                if cell.alignment is None:
                  cell.alignment = Alignment(wrap_text=True)
                else:
                  cell.alignment = Alignment(
                  horizontal=cell.alignment.horizontal,
                  vertical=cell.alignment.vertical,
                  wrap_text=True
                  )
            return max_lines

        font_size = prod_font.size if hasattr(prod_font, 'size') else 12
        line_height = font_size * 1.2  # Approximate line height
        for r in [row_top, row_bottom]:
            max_lines = get_max_lines(r, ['E', 'F', 'H', 'J'])
            sheet_814.row_dimensions[r].height = max(30, int(line_height * max_lines))



    # Automatically adjust row height to fit all text
    #for r in [row_top, row_bottom]:
     #   max_height = 15  # minimum height
      #  for col in ['E', 'F', 'H', 'J']:
       #     cell = sheet_814[f"{col}{r}"]
        #    if cell.value:
                # Estimate height: 1.2x font size per line, count lines by splitting on '\n'
         #       lines = str(cell.value).split('\n')
          #      num_lines = max(len(lines), 1)
           ##    max_height = max(max_height, height)
        #sheet_814.row_dimensions[r].height = max_height


    header = 30 #gornja sastavnica
    footer = 6 #doljnja sastavnica

    def make_header(from_sheet, header_rows, footer_rows):
    #from_sheet se samo referira na sheet excela
    # postavljanje headera kao rows
        from_sheet.print_title_rows = f'1:{header_rows}' #and f'{from_sheet.max_row-footer_rows, from_sheet.max_row}'

    # mozda bi radilo da u taj length dodam AND raspon footera al idk bi li slomilo kod

    make_header(sheet_814, header, footer)

    # 💾 Step 11: Save and download
    from openpyxl.utils import get_column_letter

    # namjesti print area na cijeli range
    # jer ako netko ne klikne "ignore print range" pri printanju
    # nece iskopirati sve redove
    max_row = sheet_814.max_row
    max_col = sheet_814.max_column
    print_area = f"A1:{get_column_letter(max_col)}{max_row}"
    sheet_814.print_area = print_area

    # Create Excel file in repo folder (you can adjust path as needed)
    output_dir = Path("output_folder")
    output_dir.mkdir(parents=True, exist_ok=True)

    # Create the target directory at the root of your repository
    mkdir -p ../excel_reports

    # Remove the existing output_folder if it's a directory
    rm -rf output_folder

    # Create a symbolic link from output_folder to the target directory
    ln -s ../excel_reports output_folder

    output_file = f"updated_shift_report_{line_input}_{date_str}.xlsx"
    full_path = output_dir / output_file
    wb.save(full_path)
    #wb.save(output_file)
    #created_files.append(output_file)
    #files.download(output_file)
    """
    # 📧 Email setup
    EMAIL_USER = os.getenv("EMAIL_USER")
    EMAIL_PASSWORD = os.getenv("EMAIL_PASSWORD")
    EMAIL_TO = os.getenv("EMAIL_TO")

    msg = EmailMessage()
    msg['Subject'] = f"Shift {shift} Reports - {selected_date.strftime('%d.%m.%Y')}"
    msg['From'] = EMAIL_USER
    msg['To'] = EMAIL_TO
    msg.set_content("Attached are the generated Excel reports.")

    for file in created_files:
      with open(file, 'rb') as f:
          file_data = f.read()
          msg.add_attachment(file_data, maintype='application',
                             subtype='vnd.openxmlformats-officedocument.spreadsheetml.sheet',
                             filename=file)

    # Send the email
    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
      smtp.login(EMAIL_USER, EMAIL_PASSWORD)
      smtp.send_message(msg)

    # 🧹 Cleanup: delete the Excel files
    for file in created_files:
      try:
        os.remove(file)
        print(f"🗑️ Deleted: {file}")
      except Exception as e:
        print(f"⚠️ Could not delete {file}: {e}")
    """
    # Git commit and push
    try:
      subprocess.run(["git", "config", "--global", "user.name", "github-actions"], check=True)
      subprocess.run(["git", "config", "--global", "user.email", "actions@github.com"], check=True)
      subprocess.run(["git", "add", str(full_path)], check=True)
      subprocess.run(["git", "commit", "-m", f"Add Excel file {filename}"], check=True)
      subprocess.run(["git", "push"], check=True)
      print("✅ Excel file committed and pushed to GitHub.")
    except subprocess.CalledProcessError as e:
      print(f"❌ Git operation failed: {e}")




In [ ]:
# Create the target directory at the root of your repository
mkdir -p ../excel_reports

# Remove the existing output_folder if it's a directory
rm -rf output_folder

# Create a symbolic link from output_folder to the target directory
ln -s ../excel_reports output_folder